<a href="https://colab.research.google.com/github/snow-The/GW190521/blob/main/homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 重力波 GW190521 數據分析與 AI 辨識專題報告

本專題旨在利用信號處理與深度學習技術，分析中等質量黑洞合併事件 GW190521。
內容涵蓋：

1. **基本項目**：數據下載、白化 (Whitening)、頻譜圖繪製。
2. **進階項目 (物理)**：牛頓力學擬合失效驗證、Ringdown 鈴振分析、廣義相對論波形擬合。
3. **進階項目 (AI)**：基於 ResNet 的深度學習訊號辨識 (含 Notch Filter 去噪與模擬數據增強)。

## 第一部分：環境安裝與設置

安裝重力波分析必要的 Python 套件 (`gwpy`, `ml4gw`, `pycbc` 等)。

In [ ]:
# === Cell 1: 環境安裝與設置 ===
!pip install "ml4gw>=0.7.10" "gwpy>=3.0" "h5py>=3.12" "torchmetrics>=1.6" "lightning>=2.4.0" "rich>=10.2.2,<14.0" "pycbc" "lalsuite"

import torch
import numpy as np
import matplotlib.pyplot as plt
from gwpy.timeseries import TimeSeries
from ml4gw.transforms import Whiten, SpectralDensity
from scipy.signal import iirnotch, filtfilt, spectrogram, butter
from scipy.optimize import curve_fit
from pycbc.waveform import get_td_waveform
import os

# 設定運算設備
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## 第二部分：數據獲取與預處理 (基本項目)

下載 GW190521 （V4）原始數據，執行白化 (Whitening) 以壓制低頻雜訊，並繪製初步頻譜圖。

In [ ]:
# === Cell 2 (最終完美版): 數據下載、白化與「宏觀 vs 微觀」雙視圖 ===
import torch
import numpy as np
import matplotlib.pyplot as plt
from gwpy.timeseries import TimeSeries
from ml4gw.transforms import Whiten, SpectralDensity
from scipy.signal import spectrogram, butter, filtfilt

# 1. 參數設定
trigger_time = 1242442967.4
sample_rate = 2048
start_time = trigger_time - 6
end_time = trigger_time + 2
psd_start = start_time - 64
psd_end = start_time
ifos = ["H1", "L1"]

# 2. 下載函數
def get_data(detectors, start, end):
    tensors = []
    print(f"Downloading data for {start} to {end}...")
    for det in detectors:
        try:
            ts = TimeSeries.fetch_open_data(det, start, end, verbose=False)
            ts = ts.resample(sample_rate)
            if np.isnan(ts.value).any():
                ts.value = np.nan_to_num(ts.value)
            tensors.append(torch.from_numpy(ts.value.copy()).float())
        except Exception as e:
            print(f"Failed to download {det}: {e}")
            return None
    return torch.stack(tensors).to(device)

# 3. 執行處理
try:
    data_tensor = get_data(ifos, start_time, end_time)
    psd_tensor = get_data(ifos, psd_start, psd_end)

    if data_tensor is not None and psd_tensor is not None:
        # 白化
        spectral_density = SpectralDensity(sample_rate=sample_rate, fftlength=2, average="median").to(device)
        whiten = Whiten(fduration=2, sample_rate=sample_rate, highpass=20).to(device)

        data_batch = data_tensor.unsqueeze(0)
        psd_batch = spectral_density(psd_tensor.unsqueeze(0).double())
        whitened_batch = whiten(data_batch, psd_batch).float()
        whitened_data = whitened_batch

        print("✅ 數據處理完成！正在繪製兩組視圖...")

        # 準備數據
        raw_strain = data_tensor[1].cpu().numpy() # L1
        whitened_strain = whitened_data[0, 1].cpu().numpy()

        # Bandpass (讓波形更乾淨)
        def bandpass_filter(data, fs, low=20, high=100):
            nyq = 0.5 * fs
            b, a = butter(4, [low/nyq, high/nyq], btype='band')
            return filtfilt(b, a, data)
        whitened_bp = bandpass_filter(whitened_strain, sample_rate)

        # 時間軸
        t_raw = np.linspace(start_time, end_time, len(raw_strain)) - trigger_time
        t_white = np.linspace(start_time + 1, end_time - 1, len(whitened_strain)) - trigger_time

        # ==========================================
        # 圖一：宏觀全景 (Global View - 8s)
        # ==========================================
        fig1, axes1 = plt.subplots(2, 2, figsize=(16, 10))
        fig1.suptitle("Figure 1: Global View (Full 8s Duration)", fontsize=16, fontweight='bold')

        # 左上：Raw Time
        axes1[0, 0].plot(t_raw, raw_strain, 'gray', alpha=0.8)
        axes1[0, 0].set_title("Raw Data (Time): Dominated by Low-Freq Noise")
        axes1[0, 0].set_ylabel("Strain"); axes1[0, 0].grid(alpha=0.3)

        # 右上：Processed Time
        axes1[0, 1].plot(t_white, whitened_bp, 'tab:blue')
        axes1[0, 1].set_title("Processed Data (Time): Noise Flattened")
        axes1[0, 1].set_ylabel("Sigma"); axes1[0, 1].grid(alpha=0.3)

        # 左下：Raw Spec
        axes1[1, 0].specgram(raw_strain, NFFT=256, Fs=sample_rate, noverlap=128, xextent=[t_raw[0], t_raw[-1]], cmap='inferno', scale='dB')
        axes1[1, 0].set_title("Raw Spectrogram: High Intensity at Low Freq")
        axes1[1, 0].set_ylabel("Freq (Hz)"); axes1[1, 0].set_yscale('log'); axes1[1, 0].set_ylim(20, 500)

        # 右下：Processed Spec
        f, t, Sxx = spectrogram(whitened_strain, fs=sample_rate, nperseg=128, noverlap=120)
        axes1[1, 1].pcolormesh(t + t_white[0], f, Sxx, shading='gouraud', cmap='viridis', vmin=0, vmax=10)
        axes1[1, 1].set_title("Processed Spectrogram: Signal too small to see")
        axes1[1, 1].set_ylabel("Freq (Hz)"); axes1[1, 1].set_ylim(20, 150)

        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()

        # ==========================================
        # 圖二：顯微聚焦 (Focused View - 0.6s)
        # ==========================================
        fig2, axes2 = plt.subplots(2, 2, figsize=(16, 10))
        fig2.suptitle("Figure 2: Focused View (Zoom-in +/- 0.3s)", fontsize=16, fontweight='bold')

        # 設定 Zoom 範圍
        zoom_range = [-0.3, 0.3]

        # 左上：Raw Time (Zoom)
        mask_raw = (t_raw > zoom_range[0]) & (t_raw < zoom_range[1])
        axes2[0, 0].plot(t_raw[mask_raw], raw_strain[mask_raw], 'gray', alpha=0.8)
        axes2[0, 0].set_title("Raw Data (Zoom): Still Random Noise")
        axes2[0, 0].set_ylabel("Strain"); axes2[0, 0].grid(alpha=0.3)

        # 右上：Processed Time (Zoom) -> 這裡訊號會現形！
        mask_white = (t_white > zoom_range[0]) & (t_white < zoom_range[1])
        axes2[0, 1].plot(t_white[mask_white], whitened_bp[mask_white], 'tab:blue', linewidth=2)
        axes2[0, 1].set_title("Processed Data (Zoom): GW190521 Waveform Revealed!")
        axes2[0, 1].set_ylabel("Sigma"); axes2[0, 1].grid(alpha=0.3)
        # 畫個虛線標示中心就好，不畫箭頭
        axes2[0, 1].axvline(0, color='r', linestyle='--', alpha=0.5)

        # 左下：Raw Spec (Zoom)
        axes2[1, 0].specgram(raw_strain, NFFT=256, Fs=sample_rate, noverlap=128, xextent=[t_raw[0], t_raw[-1]], cmap='inferno', scale='dB')
        axes2[1, 0].set_title("Raw Spectrogram (Zoom): Still Low-Freq Dominant")
        axes2[1, 0].set_ylabel("Freq (Hz)"); axes2[1, 0].set_yscale('log'); axes2[1, 0].set_ylim(20, 500)
        axes2[1, 0].set_xlim(zoom_range) # 強制 Zoom

        # 右下：Processed Spec (Zoom) -> 這裡頻譜亮點會現形！
        axes2[1, 1].pcolormesh(t + t_white[0], f, Sxx, shading='gouraud', cmap='viridis', vmin=0, vmax=12)
        axes2[1, 1].set_title("Processed Spectrogram (Zoom): Clear Merger Signal at 60Hz")
        axes2[1, 1].set_ylabel("Freq (Hz)"); axes2[1, 1].set_ylim(20, 150)
        axes2[1, 1].set_xlim(zoom_range) # 強制 Zoom

        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()

        print("✅ 雙視圖繪製完成！")
        print("圖一展示了『大海撈針』的難度，圖二展示了『數據處理』的威力。")

    else:
        print("❌ 數據下載失敗。")

except Exception as e:
    print(f"❌ Error: {e}")

## 第三部分：物理分析 (進階項目 1 & 2)

包含：

1. **牛頓力學擬合 (Inspiral)**：證明簡單牛頓模型無法解釋此事件（紅線不吻合）。
2. **廣義相對論擬合 (IMRPhenom)**：使用 PyCBC 生成相對論波形，完美重現訊號（白線吻合）。
3. **Ringdown 分析**：擬合合併後的鈴振波形，推算黑洞頻率。

In [ ]:
# === Cell 3: 物理分析 (牛頓 vs 相對論 + Ringdown) ===
if 'whitened_data' not in globals():
    print("❌ 請先執行 Cell 2！")
else:
    strain_l1 = whitened_data[0, 1].cpu().numpy()
    plot_start = start_time + 1
    plot_end = end_time - 1
    time_axis = np.linspace(plot_start, plot_end, len(strain_l1))

    plt.figure(figsize=(14, 12))

    # --- 上圖：Inspiral & Merger ---
    plt.subplot(2, 1, 1)
    fs = sample_rate
    f_vec, t_vec, Sxx = spectrogram(strain_l1, fs, nperseg=int(fs/16), noverlap=int(fs/16*0.95))
    plt.pcolormesh(t_vec + plot_start, f_vec, Sxx, shading='gouraud', cmap='viridis', vmin=0, vmax=15)

    # 牛頓力學 (紅虛線)
    G, c, M_solar = 6.674e-11, 3e8, 1.989e30
    m1, m2 = 85 * M_solar, 66 * M_solar
    chirp_mass = (m1 * m2)**(3/5) / (m1 + m2)**(1/5)
    def newtonian_freq(t, tc):
        tau = tc - t
        with np.errstate(invalid='ignore'):
            return (c**3)/(8*np.pi*G*chirp_mass) * ((5*G*chirp_mass)/(c**3*tau))**(3/8)

    t_model = np.linspace(trigger_time - 1.5, trigger_time, 1000)
    plt.plot(t_model, newtonian_freq(t_model, trigger_time+0.02), 'r--', linewidth=3, label='Newtonian')

    # 廣義相對論 (白實線)
    hp, _ = get_td_waveform(approximant="IMRPhenomXPHM", mass1=95, mass2=69, spin1z=0.7, spin2z=0.7, delta_t=1.0/fs, f_lower=20)
    hp.resize(len(hp))
    rel_time = hp.sample_times.numpy() + (trigger_time - hp.sample_times[-1])
    plt.plot(rel_time, hp.numpy() * 400 * 1e19 + 60, 'w-', linewidth=2, alpha=0.9, label='GR Waveform')

    plt.yscale('log'); plt.ylim(20, 150); plt.xlim(trigger_time-0.5, trigger_time+0.2)
    plt.title("Part 1: Newtonian vs GR", fontsize=14, fontweight='bold'); plt.legend(loc='upper left')

    # --- 下圖：Ringdown ---
    plt.subplot(2, 1, 2)
    peak_idx = np.argmax(np.abs(strain_l1[(time_axis > trigger_time-0.1) & (time_axis < trigger_time+0.1)]))
    global_peak_idx = np.where((time_axis > trigger_time-0.1))[0][0] + peak_idx

    start_fit = global_peak_idx + int(0.003 * fs)
    end_fit = global_peak_idx + int(0.05 * fs)
    t_ring = time_axis[start_fit:end_fit]
    h_ring = strain_l1[start_fit:end_fit]

    def ringdown_model(t, A, tau, f, phi):
        return A * np.exp(-(t-t[0])/tau) * np.cos(2*np.pi*f*(t-t[0]) + phi)

    try:
        popt, _ = curve_fit(ringdown_model, t_ring, h_ring, p0=[np.max(h_ring), 0.01, 65, 0])
        plt.plot(time_axis, strain_l1, 'k-', alpha=0.3)
        plt.plot(t_ring, ringdown_model(t_ring, *popt), 'r-', linewidth=2.5, label=f'Fit (f={popt[2]:.1f}Hz)')
        plt.xlim(trigger_time-0.05, trigger_time+0.1)
        plt.title(f"Part 2: Ringdown (f={popt[2]:.1f} Hz)", fontsize=14, fontweight='bold'); plt.legend()
    except:
        print("Ringdown fit failed")

    plt.tight_layout(); plt.show()


## 第四部分：深度學習訊號辨識 (進階項目 3)

使用 ResNet-18 進行「有監督學習」。

* **數據增強**：利用 `IMRPhenomXPHM` 生成模擬波形，並混合真實雜訊特徵。
* **去噪處理**：實施 Notch Filter (60Hz) 移除電源線干擾 (PhysRevLett.125.101102提到) 。
* **實戰驗證**：將模型應用於真實 GW190521 數據。

In [ ]:
# === Cell 4: AI 數據生成 (多核心加速版) ===
import concurrent.futures
from tqdm.auto import tqdm

DATA_FILE = "gw190521_dataset.pt"
NUM_SAMPLES = 12000 # 樣本數

# 工具函數
def apply_notch(data, fs=2048):
    data = np.nan_to_num(data)
    for f in [60, 120, 180]:
        b, a = iirnotch(f, 30, fs)
        data = filtfilt(b, a, data)
    return np.nan_to_num(data)

def get_ai_spec(strain, fs=2048):
    f, t, Sxx = spectrogram(strain, fs=fs, nperseg=256, noverlap=240)
    mask = (f >= 20) & (f <= 300)
    spec = np.log(Sxx[mask, :] + 1e-10)
    spec = np.clip(spec, np.mean(spec)-2.5*np.std(spec), np.mean(spec)+2.5*np.std(spec))
    return (spec - np.min(spec)) / (np.max(spec) - np.min(spec) + 1e-8)

def worker(i):
    np.random.seed(int.from_bytes(os.urandom(4), 'little'))
    N = int(2048 * 1.0)
    noise = np.random.normal(0, 1, N)
    if i % 2 == 0: # Signal
        hp, _ = get_td_waveform(approximant="IMRPhenomXPHM", mass1=np.random.uniform(50,100),
                                mass2=np.random.uniform(30,80), delta_t=1.0/2048, f_lower=20)
        hp.resize(N)
        sig = hp.numpy() * 1e19 * np.random.uniform(0.5, 4.0)
        shift = np.random.randint(0, N-len(sig) if N>len(sig) else 1)
        noise[shift:shift+len(sig)] += sig[:len(noise)-shift]
        return get_ai_spec(apply_notch(noise)), 1.0
    else: # Noise
        return get_ai_spec(apply_notch(noise)), 0.0

if not os.path.exists(DATA_FILE):
    print("🚀 開始生成數據...")
    with concurrent.futures.ProcessPoolExecutor() as ex:
        results = list(tqdm(ex.map(worker, range(NUM_SAMPLES)), total=NUM_SAMPLES))

    data, labels = zip(*results)
    torch.save({"data": torch.tensor(np.array(data)).unsqueeze(1).float(),
                "labels": torch.tensor(np.array(labels)).unsqueeze(1).float()}, DATA_FILE)
    print("✅ 數據存檔完成！")
else:
    print("⚠️ 數據檔已存在，跳過生成。")

In [ ]:
# === Cell 5 (最終修復版): 模型訓練 (含數據增強、修復 Scheduler 報錯) ===
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import models, transforms
import matplotlib.pyplot as plt
import copy
import os

# 設定運算設備 (確保與前面一致)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA_FILE = "gw190521_dataset.pt"

# 定義一個支援 Augmentation 的資料集類別
class AugmentedDataset(Dataset):
    def __init__(self, tensors, labels, transform=None):
        self.tensors = tensors
        self.labels = labels
        self.transform = transform

    def __getitem__(self, index):
        x = self.tensors[index]
        y = self.labels[index]
        if self.transform:
            # x 的形狀是 (1, Freq, Time)，Transform 會自動處理
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.tensors)

if os.path.exists(DATA_FILE):
    # 1. 載入數據
    saved = torch.load(DATA_FILE)
    full_data = saved["data"]
    full_labels = saved["labels"]

    # 檢查數據平衡性
    sig_count = (full_labels == 1.0).sum().item()
    print(f"📊 數據載入成功: 總數 {len(full_labels)} | 訊號 {sig_count} | 雜訊 {len(full_labels)-sig_count}")

    # 2. 定義數據增強 (Data Augmentation) - 這裡就是你要的特訓！
    train_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(p=0.5), # 這個保留，很有用
        # 暫時關掉旋轉和遮擋，先求有再求好
        # transforms.RandomApply([transforms.RandomRotation(degrees=5)], p=0.3),
        # transforms.RandomErasing(p=0.2, scale=(0.02, 0.1), ratio=(0.3, 3.3))
    ])

    # 3. 分割訓練與驗證集
    total_len = len(full_data)
    train_len = int(0.8 * total_len)
    val_len = total_len - train_len

    # 隨機打亂索引
    generator = torch.Generator().manual_seed(42) # 固定種子以求重現
    perm = torch.randperm(total_len, generator=generator)
    train_idx = perm[:train_len]
    val_idx = perm[train_len:]

    # 建立 Dataset (只有訓練集套用 transform)
    train_ds = AugmentedDataset(full_data[train_idx], full_labels[train_idx], transform=train_transform)
    val_ds = AugmentedDataset(full_data[val_idx], full_labels[val_idx], transform=None)

    # 建立 DataLoader (Batch Size 設為 32 或 64)
    train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
    val_dl = DataLoader(val_ds, batch_size=64)

    # 4. 定義模型 (ResNet18)
    model = models.resnet18(weights=None)
    # 修改第一層 (輸入通道改為 1)
    model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
    # 修改最後一層 (輸出改為 1)
    model.fc = nn.Sequential(
        nn.Dropout(0.5), # Dropout 防止過擬合
        nn.Linear(model.fc.in_features, 1)
    )
    model = model.to(device)

    # 優化器與 Loss
    # 使用 AdamW 加上 Weight Decay 來抑制過擬合
    opt = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-3)
    crit = nn.BCEWithLogitsLoss()

    # 學習率調度器 (修正版：移除了 verbose=True)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(opt, mode='max', factor=0.5, patience=3)

    # 5. 訓練迴圈
    hist = {'train_loss':[], 'val_loss':[], 'val_acc':[]}
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())

    epochs = 15
    print(f"🚀 開始訓練美少女 AI (ResNet18) - 共 {epochs} 輪...")
    print("-" * 70)

    for epoch in range(epochs):
        model.train()
        run_loss = 0

        # 訓練階段
        for x, y in train_dl:
            x, y = x.to(device), y.to(device)
            opt.zero_grad()
            out = model(x)
            loss = crit(out, y)
            loss.backward()
            opt.step()
            run_loss += loss.item()

        # 驗證階段
        model.eval()
        val_loss, correct = 0, 0
        with torch.no_grad():
            for x, y in val_dl:
                x, y = x.to(device), y.to(device)
                out = model(x)
                val_loss += crit(out, y).item()
                # 預測 > 0.5 (經過 Sigmoid 後) 為訊號
                predicted = (torch.sigmoid(out) > 0.5).float()
                correct += (predicted == y).sum().item()

        # 統計數據
        epoch_loss = run_loss / len(train_dl)
        epoch_val_loss = val_loss / len(val_dl)
        epoch_acc = correct / len(val_ds)
        current_lr = opt.param_groups[0]['lr']

        hist['train_loss'].append(epoch_loss)
        hist['val_loss'].append(epoch_val_loss)
        hist['val_acc'].append(epoch_acc)

        # 更新 Scheduler (根據驗證準確率)
        scheduler.step(epoch_acc)

        # 顯示進度與儲存最佳模型
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            print(f"Epoch {epoch+1:02d} | 🟢 Train: {epoch_loss:.4f} | Val: {epoch_val_loss:.4f} | Acc: {epoch_acc:.2%} | LR: {current_lr:.1e} (New Best!)")
        else:
            print(f"Epoch {epoch+1:02d} | ⚪ Train: {epoch_loss:.4f} | Val: {epoch_val_loss:.4f} | Acc: {epoch_acc:.2%} | LR: {current_lr:.1e}")

    # 6. 訓練結束，載入最佳權重
    print("-" * 70)
    print(f"🏆 訓練完成！最佳驗證準確率: {best_acc:.2%}")
    model.load_state_dict(best_model_wts)

    # 繪製圖表
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(hist['train_loss'], label='Train Loss', color='tab:blue')
    plt.plot(hist['val_loss'], label='Val Loss', color='tab:orange')
    plt.title("Loss Curve (Lower is Better)")
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend(); plt.grid(True, alpha=0.3)

    plt.subplot(1, 2, 2)
    plt.plot(hist['val_acc'], label='Val Accuracy', color='tab:green')
    plt.title("Accuracy Curve (Higher is Better)")
    plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.legend(); plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

else:
    print("❌ 找不到數據檔，請先執行 Cell 4 生成數據！")

In [ ]:
# === Cell 6: 最終 AI 驗收 (Signal vs Noise) ===
# 確保使用與訓練一致的預處理
def get_ai_spec(strain, fs=2048):
    f, t, Sxx = spectrogram(strain, fs=fs, nperseg=256, noverlap=240)
    mask = (f >= 20) & (f <= 300)
    spec = np.log(Sxx[mask, :] + 1e-10)
    spec = np.clip(spec, np.mean(spec)-2.5*np.std(spec), np.mean(spec)+2.5*np.std(spec))
    return (spec - np.min(spec)) / (np.max(spec) - np.min(spec) + 1e-8), t, f[mask]

# 抓取真實數據
idx_sig = int(5.0 * sample_rate)
idx_noise = int(2.0 * sample_rate)
hw = int(0.5 * sample_rate)

sig_data = whitened_data[0, 1, idx_sig-hw:idx_sig+hw].cpu().numpy()
noise_data = whitened_data[0, 1, idx_noise-hw:idx_noise+hw].cpu().numpy()

# 預測
model.eval()
spec_sig, _, f_axis = get_ai_spec(sig_data)
spec_noise, _, _ = get_ai_spec(noise_data)

with torch.no_grad():
    score_sig = torch.sigmoid(model(torch.tensor(spec_sig).unsqueeze(0).unsqueeze(0).float().to(device))).item()
    score_noise = torch.sigmoid(model(torch.tensor(spec_noise).unsqueeze(0).unsqueeze(0).float().to(device))).item()

# 繪圖
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
ax[0].imshow(spec_sig, origin='lower', aspect='auto', cmap='viridis', extent=[-0.5, 0.5, f_axis[0], f_axis[-1]])
ax[0].set_title(f"Target: Signal\nConf: {score_sig:.4f}", color='green', fontweight='bold')
ax[0].axvline(0, color='r', linestyle='--')

ax[1].imshow(spec_noise, origin='lower', aspect='auto', cmap='viridis', extent=[-0.5, 0.5, f_axis[0], f_axis[-1]])
ax[1].set_title(f"Control: Noise\nConf: {score_noise:.4f}", color='black', fontweight='bold')

plt.show()
print(f"🚀 AI 判斷：訊號信心度 {score_sig:.2%} | 雜訊誤判率 {score_noise:.2%}")

In [ ]:
# === Cell 7 (修正版): 製作波形掃描動畫並存檔 (MP4/GIF) ===
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import rc
import numpy as np

# 1. 解除動畫大小限制 (設為 100MB)
plt.rcParams['animation.embed_limit'] = 100.0
# 設定 ffmpeg (用於生成 MP4)
rc('animation', html='jshtml')

print("正在製作並儲存動畫，請稍候...")

# 2. 準備數據
# 為了避免檔案過大，我們只取最精彩的 1 秒鐘 (Trigger 前後 0.5s)
center_idx = int(5.0 * sample_rate)
window = int(0.5 * sample_rate)
data_anim = whitened_data[0, 1, center_idx-window : center_idx+window].cpu().numpy()

# 簡單濾波讓線條好看點
def apply_notch_anim(data, fs=2048):
    from scipy.signal import iirnotch, filtfilt
    data = np.nan_to_num(data)
    for f in [60, 120, 180]:
        b, a = iirnotch(f, 30, fs)
        data = filtfilt(b, a, data)
    return data

data_anim = apply_notch_anim(data_anim, sample_rate)
t_anim = np.linspace(-0.5, 0.5, len(data_anim))

# 3. 設定畫布 (調整 figsize 和 DPI 以控制檔案大小)
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 6), dpi=100, sharex=True)

# 上圖：波形
line, = ax1.plot([], [], 'w-', linewidth=1.5)
ax1.set_xlim(-0.5, 0.2)
ax1.set_ylim(-6, 6)
ax1.set_facecolor('black')
ax1.set_ylabel('Strain', color='white')
ax1.set_title('GW190521 Merger Event', fontsize=14)
ax1.grid(True, alpha=0.3, color='gray')

# 下圖：頻譜圖
from scipy.signal import spectrogram
f, t, Sxx = spectrogram(data_anim, fs=sample_rate, nperseg=128, noverlap=120)
t = t - 0.5 # 校正時間
mesh = ax2.pcolormesh(t, f, Sxx, shading='gouraud', cmap='viridis', vmin=0, vmax=10)
ax2.set_ylabel('Frequency (Hz)')
ax2.set_xlabel('Time (s)')
ax2.set_ylim(20, 150)
ax2.set_facecolor('black')

# 掃描線
vline1 = ax1.axvline(x=-0.5, color='r', linestyle='--')
vline2 = ax2.axvline(x=-0.5, color='r', linestyle='--')

plt.tight_layout()

# 4. 更新函數
def update(frame):
    current_time = t_anim[frame]
    # 更新波形
    line.set_data(t_anim[:frame], data_anim[:frame])
    # 更新掃描線
    vline1.set_xdata([current_time])
    vline2.set_xdata([current_time])
    return line, vline1, vline2

# 5. 生成動畫 (降低採樣率 Step=4 以縮小體積)
step = 4
frames = range(0, len(t_anim), step)
ani = animation.FuncAnimation(fig, update, frames=frames, interval=20, blit=True)

# === 關鍵修改：存成獨立檔案 ===
# 儲存 MP4 (PPT 首選，畫質好體積小)
print("正在儲存為 gw190521.mp4 ...")
ani.save('gw190521.mp4', writer='ffmpeg', fps=30)

# 儲存 GIF (備用，相容性高但體積大)
print("正在儲存為 gw190521.gif ...")
ani.save('gw190521.gif', writer='pillow', fps=15)

plt.close()
print("✅ 動畫製作完成！")
print("📥 請查看左側檔案欄 (Files)，下載 'gw190521.mp4' 或 'gw190521.gif' 放入投影片。")